## Music Genre Classifier With TensorFlow

The objective of this project is to classify 30 sec wav files by genre using a TensorFlow CNN model. The GTZAN dataset can be found here:

https://www.kaggle.com/andradaolteanu/gtzan-dataset-music-genre-classification

To classify audio samples, we will preprocess them by calculating their MFCC, which is a temporal representation of the energy for each perceived frequency band. In this case, we are choosing 13 bands.

In [1]:
import os
import json
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from keras.models import load_model

import wandb
from wandb.keras import WandbCallback

In [2]:
sweep_config = {
    'method': 'random', #grid
    'metric': {
      'name': 'val_acc',
      'goal': 'maximize'   
    },
    'parameters': {
        'epochs': {
            'values': [20, 30]
        },
        'batch_size': {
            'values': [32, 48]
        },
        'dropout': {
            'values': [0.2, 0.3, 0.4]
        },
        'conv_layer_size': {
            'values': [32, 64, 128]
        },
        'hidden_layer_size': {
            'values': [64, 128]
        },
        'learning_rate': {
            'values': [0.0005, 0.001, 0.005]
        },
    }
}

In [3]:
sweep_id = wandb.sweep(sweep_config, entity="msaintfelix", project="Genre-Classifier-Optimization")

Create sweep with ID: auytih7y
Sweep URL: https://wandb.ai/msaintfelix/Genre-Classifier-Optimization/sweeps/auytih7y


In [4]:
# Dataset location
SOURCE_PATH = 'Dataset/genres_original/'
LOCAL_MODEL_DIR = 'Model/'

# Path to labels and processed data file, json format.
JSON_PATH = 'data.json'

# Sampling rate.
sr = 22050

# Let's make sure all files have the same amount of samples, pick a duration right under 30 seconds.
TOTAL_SAMPLES = 29 * sr

# The dataset contains 999 files. Let's make it bigger. 
# X amount of slices => X times more training examples.
NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

In [5]:
def preprocess_data(source_path, json_path):

    # Let's create a dictionary of labels and processed data.
    mydict = {
        "labels": [],
        "mfcc": []
        }

    # Let's browse each file, slice it and generate the 13 band mfcc for each slice.
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(source_path)):

        for file in filenames:
            song, sr = librosa.load(os.path.join(dirpath, file), duration=29)

            for s in range(NUM_SLICES):
                start_sample = SAMPLES_PER_SLICE * s
                end_sample = start_sample + SAMPLES_PER_SLICE
                mfcc = librosa.feature.mfcc(y=song[start_sample:end_sample], sr=sr, n_mfcc=13)
                mfcc = mfcc.T
                mydict["labels"].append(i-1)
                mydict["mfcc"].append(mfcc.tolist())

    # Let's write the dictionary in a json file.    
    with open(json_path, 'w') as f:
        json.dump(mydict, f)
    f.close()

In [6]:
def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()

    # Let's load our data into numpy arrays for TensorFlow compatibility.
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [7]:
def prepare_datasets(inputs, targets, split_size):
    
    # Creating a validation set and a test set with fixed random_state
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size, random_state=42)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size, random_state=42)
    
    # Our CNN model expects 3D input shape.
    inputs_train = inputs_train[..., np.newaxis]
    inputs_val = inputs_val[..., np.newaxis]
    inputs_test = inputs_test[..., np.newaxis]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [8]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'epochs': 20,
        'batch_size': 32,
        'learning_rate': 1e-3,
        'dropout': 0.3,
        'hidden_layer_size': 64,
        'conv_layer_size': 32
        }
    
    genre_dict = {
        0 : "blues",
        1 : "classical",
        2 : "country",
        3 : "disco",
        4 : "hiphop",
        5 : "jazz",
        6 : "metal",
        7 : "pop",
        8 : "reggae",
        9 : "rock",
        }
        
    # Initialize a new wandb run
    run = wandb.init(config=config_defaults)
    
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config

    #preprocess_data(source_path=SOURCE_PATH, json_path=JSON_PATH)
    
    inputs, targets = load_data(json_path=JSON_PATH)     
    Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)

    input_shape = (Xtrain.shape[1], Xtrain.shape[2], 1)
    
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Conv2D(config.conv_layer_size, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(config.conv_layer_size, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        
        tf.keras.layers.Conv2D(config.conv_layer_size, (2,2), activation='relu'),
        tf.keras.layers.MaxPooling2D((3,3), strides=(2,2), padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(config.dropout),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(config.hidden_layer_size, activation='relu'), 
        tf.keras.layers.Dense(len(np.unique(targets)), activation='softmax')
    ])

    # Selection of the optimizer, loss type and metrics for performance evaluation.
    model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=config.learning_rate),
                     loss='sparse_categorical_crossentropy',
                     metrics = ['acc']
                     )

    model.summary()

    # Training the model
    history = model.fit(Xtrain, ytrain,
                        validation_data=(Xval, yval),
                        epochs=config.epochs,
                        batch_size=config.batch_size,
                        callbacks=[WandbCallback()]
                        )
    
    # Store trained model artifacts
    model_artifact_name = "CNN_Genre_Classifier_" + run.id
    model_artifact = wandb.Artifact(model_artifact_name, type='model')
    model.save('Model/my_model.h5')
    model_artifact.add_dir(LOCAL_MODEL_DIR)
    run.log_artifact(model_artifact)
    
    # log the confusion matrix for validation data
    pred = model.predict(Xval)
    predictions = pred.argmax(axis=1)
    wandb.log({"conf_mat" : wandb.plot.confusion_matrix(
                            probs=None,
                            preds=predictions.tolist(), 
                            y_true=yval.tolist(),
                            class_names=genre_dict)
              })

In [9]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: 81846d9n with config:
wandb: 	batch_size: 32
wandb: 	conv_layer_size: 128
wandb: 	dropout: 0.4
wandb: 	epochs: 30
wandb: 	hidden_layer_size: 128
wandb: 	learning_rate: 0.001
wandb: Currently logged in as: msaintfelix. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 123, 11, 128)      1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 6, 128)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 6, 128)        512       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 4, 128)        147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 2, 128)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 2, 128)        512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 1, 128)        6

wandb: Adding directory to artifact (/Users/msf/GitHub/TensorFlow_MusicGenre_Classifier/wandb/run-20220901_232621-81846d9n/files/model-best)... Done. 0.0s


Epoch 2/30
200/200 [==============================] - 38s 189ms/step - loss: 1.2731 - acc: 0.5803 - val_loss: 1.3160 - val_acc: 0.6286
INFO:tensorflow:Assets written to: /Users/msf/GitHub/TensorFlow_MusicGenre_Classifier/wandb/run-20220901_232621-81846d9n/files/model-best/assets


...

wandb: Adding directory to artifact (/Users/msf/GitHub/TensorFlow_MusicGenre_Classifier/wandb/run-20220902_110419-mitmn8vd/files/model-best)... Done. 0.0s


Epoch 11/30
200/200 [==============================] - 40s 198ms/step - loss: 0.2629 - acc: 0.9083 - val_loss: 0.7736 - val_acc: 0.7938
Epoch 12/30
200/200 [==============================] - 35s 176ms/step - loss: 0.2175 - acc: 0.9286 - val_loss: 0.7576 - val_acc: 0.7818
Epoch 13/30
200/200 [==============================] - 35s 174ms/step - loss: 0.2115 - acc: 0.9285 - val_loss: 0.8051 - val_acc: 0.7873
Epoch 14/30
200/200 [==============================] - 40s 202ms/step - loss: 0.1919 - acc: 0.9384 - val_loss: 0.8065 - val_acc: 0.7828
Epoch 15/30
200/200 [==============================] - 48s 239ms/step - loss: 0.1844 - acc: 0.9423 - val_loss: 0.8353 - val_acc: 0.7833
Epoch 16/30
200/200 [==============================] - 59s 296ms/step - loss: 0.1634 - acc: 0.9479 - val_loss: 0.7788 - val_acc: 0.8043
Epoch 17/30
200/200 [==============================] - 40s 200ms/step - loss: 0.1534 - acc: 0.9482 - val_loss: 0.9502 - val_acc: 0.7753
Epoch 18/30
200/200 [===========================

wandb: Adding directory to artifact (./Model)... Done. 0.0s


acc,▁▃▄▅▅▆▆▆▇▇▇▇▇▇████████████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
loss,█▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▅▆▇▆▅▆▆▇▇▇▇▇▇█▇▇▇█▇▇▇██▆▇███
val_loss,█▅▃▂▂▃▄▃▃▁▁▁▂▂▂▁▂▃▃▁▂▄▃▂▂▆▄▂▂▂
GFLOPs,0.03933
acc,0.96653
best_epoch,9
best_val_loss,0.70696
epoch,29
loss,0.11268


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
